# MGIMO intensive

## Relational databases practice - 2

### 1. Libraries

In [ ]:
%load_ext sql
# there is a bug with `sql` magic, have to fix it
#%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
import os
import sys

### 2. Create database and tables

#### 2.1. Create database

You may need a [PostgreSQL manual](https://www.postgresql.org/docs/current/sql-createdatabase.html) to create a table and read about [data types](https://www.postgresql.org/docs/current/datatype.html)

In [ ]:
%%sql postgresql:///jovyan
    SELECT * FROM pg_database

In [ ]:
%%sql postgresql:///jovyan
    CREATE DATABASE test

In [ ]:
%%sql postgresql:///test
    SELECT tablename AS table FROM pg_tables WHERE tablename !~ '^(pg_|sql_)'

#### 2.2. Create tables

#### 2.2.1. Regions

Data sample can be taken from previous notebook:

| region_id | region_name |
| --- | --- |
| 1 | Алтайский край |
| 3 | Краснодарский край |
| 4 | Красноярский край |
| 5 | Приморский край |
| 7 | Ставропольский край |

Prompt also gave us a set of SQL queries to create tables:

```SQL
-- 1. Справочник регионов
CREATE TABLE dim_region (
    region_id INTEGER PRIMARY KEY,
    region_name VARCHAR(200) NOT NULL
);

-- 2. Справочник разделов показателей
CREATE TABLE dim_indicator_section (
    indicator_section_code INTEGER PRIMARY KEY,
    indicator_section VARCHAR(200) NOT NULL
);

-- 3. Справочник показателей
CREATE TABLE dim_indicator (
    indicator_code VARCHAR(50) PRIMARY KEY,
    indicator_name VARCHAR(500) NOT NULL,
    indicator_section_code INTEGER REFERENCES dim_indicator_section(indicator_section_code)
);

-- 4. Справочник муниципальных образований
CREATE TABLE dim_municipality (
    municipality_sk INTEGER PRIMARY KEY,
    oktmo INTEGER NOT NULL,
    region_id INTEGER REFERENCES dim_region(region_id),
    mun_level VARCHAR(100),
    mun_district VARCHAR(200),
    municipality_name VARCHAR(200),
    mun_type VARCHAR(100),
    mun_type_oktmo VARCHAR(100),
    oktmo_stable VARCHAR(50),
    oktmo_history VARCHAR(500),
    oktmo_year_from INTEGER,
    oktmo_year_to INTEGER,
    is_current BOOLEAN,
    UNIQUE(oktmo, oktmo_year_from)
);

-- 5. Таблица фактов
CREATE TABLE fact_indicator (
    fact_id BIGINT PRIMARY KEY,
    municipality_sk INTEGER REFERENCES dim_municipality(municipality_sk),
    indicator_code VARCHAR(50) REFERENCES dim_indicator(indicator_code),
    year INTEGER NOT NULL,
    indicator_value NUMERIC(20, 2),
    indicator_unit VARCHAR(50),
    indicator_period VARCHAR(100),
    comment TEXT,
    zagr VARCHAR(10),
    loaded_at TIMESTAMP
);

-- Индексы для производительности
CREATE INDEX idx_fact_municipality ON fact_indicator(municipality_sk);
CREATE INDEX idx_fact_indicator ON fact_indicator(indicator_code);
CREATE INDEX idx_fact_year ON fact_indicator(year);
CREATE INDEX idx_municipality_oktmo ON dim_municipality(oktmo);
```

In [ ]:
%%sql postgresql:///test
    CREATE TABLE dim_region (
    region_id INTEGER PRIMARY KEY,
    region_name VARCHAR(200) NOT NULL
);

In [ ]:
!ls -la

In [ ]:
!pwd

In [ ]:
%%sql postgresql:///test
    COPY dim_region FROM '/home/jovyan/mgimo_intensive/2_data/dim_region.csv' DELIMITER ',' CSV HEADER

In [ ]:
# If you need to delete table
#!psql -d test -c 'DROP TABLE dim_region'

In [ ]:
%%sql postgresql:///test
    SELECT * FROM dim_region LIMIT 5

#### 2.2.2. Indicator section

In [ ]:
%%sql postgresql:///test
    CREATE TABLE dim_indicator_section (
    indicator_section_code INTEGER PRIMARY KEY,
    indicator_section VARCHAR(200) NOT NULL
);

In [ ]:
%%sql postgresql:///test
    COPY dim_indicator_section FROM '/home/jovyan/mgimo_intensive/2_data/dim_indicator_section.csv' DELIMITER ',' CSV HEADER

In [ ]:
%%sql postgresql:///test
    SELECT * FROM dim_indicator_section LIMIT 5

#### 2.2.3. Indicators

In [ ]:
%%sql postgresql:///test
    CREATE TABLE dim_indicator (
    indicator_code VARCHAR(50) PRIMARY KEY,
    indicator_name VARCHAR(500) NOT NULL,
    indicator_section_code INTEGER REFERENCES dim_indicator_section(indicator_section_code)
);

In [ ]:
%%sql postgresql:///test
    COPY dim_indicator FROM '/home/jovyan/mgimo_intensive/2_data/dim_indicator.csv' DELIMITER ',' CSV HEADER

In [ ]:
%%sql postgresql:///test
    SELECT * FROM dim_indicator LIMIT 5;

#### 2.2.4. Indicators

In [ ]:
%%sql postgresql:///test
    CREATE TABLE dim_municipality (
    municipality_sk INTEGER PRIMARY KEY,
    oktmo INTEGER NOT NULL,
    region_id INTEGER REFERENCES dim_region(region_id),
    mun_level VARCHAR(100),
    mun_district VARCHAR(200),
    municipality_name VARCHAR(200),
    mun_type VARCHAR(100),
    mun_type_oktmo VARCHAR(100),
    oktmo_stable VARCHAR(50),
    oktmo_history VARCHAR(500),
    oktmo_year_from INTEGER,
    oktmo_year_to INTEGER,
    is_current BOOLEAN
);

In [ ]:
%%sql postgresql:///test
    COPY dim_municipality FROM '/home/jovyan/mgimo_intensive/2_data/dim_municipality.csv' DELIMITER ',' CSV HEADER

In [ ]:
# If you need to delete table
#!psql -d test -c 'DROP TABLE dim_municipality'

In [ ]:
%%sql postgresql:///test
    SELECT * FROM dim_municipality LIMIT 5;

#### 2.2.5. Facts

In [ ]:
%%sql postgresql:///test
    CREATE TABLE fact_indicator (
    fact_id BIGINT PRIMARY KEY,
    municipality_sk INTEGER REFERENCES dim_municipality(municipality_sk),
    indicator_code VARCHAR(50) REFERENCES dim_indicator(indicator_code),
    year INTEGER NOT NULL,
    indicator_value NUMERIC(20, 2),
    indicator_unit VARCHAR(50),
    indicator_period VARCHAR(100),
    comment TEXT,
    zagr VARCHAR(256),
    loaded_at TIMESTAMP
);

In [ ]:
%%sql postgresql:///test
    COPY fact_indicator FROM '/home/jovyan/mgimo_intensive/2_data/fact_indicator.csv' DELIMITER ',' CSV HEADER

In [ ]:
# If you need to delete table
#!psql -d test -c 'DROP TABLE fact_indicator'

### 3. Tests

In [ ]:
%%sql postgresql:///test
    -- Проверка количества записей (сравнение с исходным DataFrame)
    SELECT 'fact_indicator' as table_name, COUNT(*) as record_count FROM fact_indicator
    UNION ALL
    SELECT 'dim_region', COUNT(*) FROM dim_region
    UNION ALL
    SELECT 'dim_indicator_section', COUNT(*) FROM dim_indicator_section
    UNION ALL
    SELECT 'dim_indicator', COUNT(*) FROM dim_indicator
    UNION ALL
    SELECT 'dim_municipality', COUNT(*) FROM dim_municipality
    ORDER BY table_name;

In [ ]:
%%sql postgresql:///test
    -- Топ-10 показателей по сумме значений
    SELECT i.indicator_code, 
           i.indicator_name,
           s.indicator_section,
           COUNT(*) as record_count,
           SUM(f.indicator_value) as total_value,
           AVG(f.indicator_value) as avg_value
    FROM fact_indicator f
    JOIN dim_indicator i ON f.indicator_code = i.indicator_code
    JOIN dim_indicator_section s ON i.indicator_section_code = s.indicator_section_code
    GROUP BY i.indicator_code, i.indicator_name, s.indicator_section
    ORDER BY total_value DESC
    LIMIT 10;

In [ ]:
%%sql postgresql:///test
    -- Динамика показателей по региону
    SELECT 
        r.region_name,
        f.year,
        i.indicator_name,
        SUM(f.indicator_value) as total_value
    FROM fact_indicator f
    JOIN dim_municipality m ON f.municipality_sk = m.municipality_sk
    JOIN dim_region r ON m.region_id = r.region_id
    JOIN dim_indicator i ON f.indicator_code = i.indicator_code
    WHERE r.region_name = 'Алтайский край'
      AND i.indicator_code = 'Y48055001'
    GROUP BY r.region_name, f.year, i.indicator_name
    ORDER BY f.year;    

In [ ]:
%%sql postgresql:///test
    -- Сравнение муниципальных районов внутри региона
    SELECT 
        m.mun_district,
        m.municipality_name,
        f.year,
        AVG(f.indicator_value) as avg_value
    FROM fact_indicator f
    JOIN dim_municipality m ON f.municipality_sk = m.municipality_sk
    JOIN dim_region r ON m.region_id = r.region_id
    JOIN dim_indicator i ON f.indicator_code = i.indicator_code
    WHERE r.region_name = 'Алтайский край'
      AND i.indicator_code = 'Y48055001'
      AND f.year = 2024
    GROUP BY m.mun_district, m.municipality_name, f.year
    ORDER BY avg_value DESC;